![erd](./images/ERD-week5-exercise.png)

## Instruction

1. ใช้ project myshop เดิมจากของ WEEK 4 มาทำต่อได้เลยครับ (ใครที่หา project ของ WEEK 4 ไม่เจอแล้ว ให้กลับไปทำตามขั้นตอนใน week4_exercises.ipynb)
2. ใช้ database `shop` เดิมได้เลยเช่นกันครับ

**หมายเหตุ: ถ้านักศึกษาใช้ database เดิมจาก WEEK4 ซึ่งนักศึกษาเคยเพิ่มหรือแก้ไขข้อมูลบางส่วนแล้ว ดังนั้น output อาจจะไม่ตรงกับในตัวอย่างทั้งหมดไม่ต้องตกใจนะครับ**

In [2]:
import os
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = "true"

# import modules
from shop.models import *
from django.db.models import F, Q, Value as V, Count, OuterRef, Subquery
from django.db.models.functions import Concat, JSONObject
import json
from datetime import datetime

### 1. annotate(), F()

1.1 ให้นักศึกษา query ข้อมูล `Payment` โดยให้เพิ่ม field ราคาที่ลบกับส่วนลดแล้ว โดยกำหนดให้ชื่อ field ว่า "after_discount_price" โดยใช้แสดงข้อมูล 10 ตัวแรกเรียงตาม "after_discount_price" จากมากไปน้อย (0.25 คะแนน)

**หมายเหตุ: จะต้องใช้ annotate() นะครับ ให้เอา `Payment.price` - `Payment.discount`**

ตัวอย่าง Output

```
ID: 92, PRICE: 1200500.00, DISCOUNT 29433.25, AFTER_DISCOUNT 1171066.75
ID: 82, PRICE: 1200280.00, DISCOUNT 46229.40, AFTER_DISCOUNT 1154050.60
ID: 137, PRICE: 1200690.00, DISCOUNT 71407.25, AFTER_DISCOUNT 1129282.75
ID: 105, PRICE: 1200390.00, DISCOUNT 105019.11, AFTER_DISCOUNT 1095370.89
ID: 45, PRICE: 1218900.00, DISCOUNT 126859.95, AFTER_DISCOUNT 1092040.05
ID: 7, PRICE: 1201200.00, DISCOUNT 113446.20, AFTER_DISCOUNT 1087753.80
ID: 18, PRICE: 1202190.00, DISCOUNT 121922.64, AFTER_DISCOUNT 1080267.36
ID: 77, PRICE: 379000.00, DISCOUNT 19397.00, AFTER_DISCOUNT 359603.00
ID: 127, PRICE: 320450.00, DISCOUNT 14578.90, AFTER_DISCOUNT 305871.10
ID: 125, PRICE: 320399.00, DISCOUNT 17939.55, AFTER_DISCOUNT 302459.45
```

In [3]:
payment = Payment.objects.annotate(after_discount_price=F('price')-F('discount')).order_by("-after_discount_price")[:10]
for p in payment:
    print(F"ID: {p.id} PRICE: {p.price}, DISCOUNT: {p.discount}, AFTER_DISCOUNT {p.after_discount_price}")

ID: 92 PRICE: 1200500.00, DISCOUNT: 29433.25, AFTER_DISCOUNT 1171066.75
ID: 82 PRICE: 1200280.00, DISCOUNT: 46229.40, AFTER_DISCOUNT 1154050.60
ID: 137 PRICE: 1200690.00, DISCOUNT: 71407.25, AFTER_DISCOUNT 1129282.75
ID: 105 PRICE: 1200390.00, DISCOUNT: 105019.11, AFTER_DISCOUNT 1095370.89
ID: 45 PRICE: 1218900.00, DISCOUNT: 126859.95, AFTER_DISCOUNT 1092040.05
ID: 7 PRICE: 1201200.00, DISCOUNT: 113446.20, AFTER_DISCOUNT 1087753.80
ID: 18 PRICE: 1202190.00, DISCOUNT: 121922.64, AFTER_DISCOUNT 1080267.36
ID: 77 PRICE: 379000.00, DISCOUNT: 19397.00, AFTER_DISCOUNT 359603.00
ID: 127 PRICE: 320450.00, DISCOUNT: 14578.90, AFTER_DISCOUNT 305871.10
ID: 125 PRICE: 320399.00, DISCOUNT: 17939.55, AFTER_DISCOUNT 302459.45


1.2 ต่อเนื่องจากข้อ 1.1 ให้ filter เฉพาะข้อมูล `Payment` ที่มี "after_discount_price" มากกว่า 500,000 (0.25 คะแนน)

ตัวอย่าง Output

```
ID: 92, PRICE: 1200500.00, DISCOUNT 29433.25, AFTER_DISCOUNT 1171066.75
ID: 82, PRICE: 1200280.00, DISCOUNT 46229.40, AFTER_DISCOUNT 1154050.60
ID: 137, PRICE: 1200690.00, DISCOUNT 71407.25, AFTER_DISCOUNT 1129282.75
ID: 105, PRICE: 1200390.00, DISCOUNT 105019.11, AFTER_DISCOUNT 1095370.89
ID: 45, PRICE: 1218900.00, DISCOUNT 126859.95, AFTER_DISCOUNT 1092040.05
ID: 7, PRICE: 1201200.00, DISCOUNT 113446.20, AFTER_DISCOUNT 1087753.80
ID: 18, PRICE: 1202190.00, DISCOUNT 121922.64, AFTER_DISCOUNT 1080267.36
```

In [16]:
payment = Payment.objects.annotate(after_discount_price=F('price')-F('discount')).filter(after_discount_price__gt=500000)
for p in payment:
    print(f"ID: {p.id} PRICE: {p.price} DISCOUNT: {p.discount} AFTER_DISCOUNT: {p.after_discount_price}")

ID: 7 PRICE: 1201200.00 DISCOUNT: 113446.20 AFTER_DISCOUNT: 1087753.80
ID: 18 PRICE: 1202190.00 DISCOUNT: 121922.64 AFTER_DISCOUNT: 1080267.36
ID: 45 PRICE: 1218900.00 DISCOUNT: 126859.95 AFTER_DISCOUNT: 1092040.05
ID: 82 PRICE: 1200280.00 DISCOUNT: 46229.40 AFTER_DISCOUNT: 1154050.60
ID: 92 PRICE: 1200500.00 DISCOUNT: 29433.25 AFTER_DISCOUNT: 1171066.75
ID: 105 PRICE: 1200390.00 DISCOUNT: 105019.11 AFTER_DISCOUNT: 1095370.89
ID: 137 PRICE: 1200690.00 DISCOUNT: 71407.25 AFTER_DISCOUNT: 1129282.75


1.3 ให้นักศึกษาเรียงลำดับข้อมูลลูกค้า (`Customer`) โดยเรียงลำดับตามลำดับตัวอักษร `น้อยไปมาก` (A-Z) จากชื่อเต็มของลูกค้า (`full_name`) โดยแสดง 5 คนแรก (0.5 คะแนน)

**Hint:** Field `full_name` นั้นจะต้องถูก annotate ขึ้นมาโดยการนำ `first_name` มาต่อกับ `last_name` โดยใช้ `Concat(*expressions, **extra)` 

**Hint:** แปลง object เป็น dict ใช้ `values()` [doc](https://docs.djangoproject.com/en/5.0/ref/models/querysets/#values)

```python
>>> Blog.objects.filter(name__startswith="Beatles").values()
<QuerySet [{'id': 1, 'name': 'Beatles Blog', 'tagline': 'All the latest Beatles news.'}]>
```

**Hint:** อยาก print dictionary สวยๆ ใช้ `json.dumps`

```python
print(json.dumps(dictionary, indent=4, sort_keys=False))
```

[doc](https://docs.djangoproject.com/en/5.0/ref/models/database-functions/#concat)

ตัวอย่าง Output 

```python
{
    "id": 17,
    "email": "anantaya.deena@gmail.com",
    "address": {
        "district": "Yan Nawa",
        "location": "60 Thanon Chan Road",
        "province": "Bangkok",
        "postal_code": 10120
    },
    "full_name": "Anantaya Tontong"
}
{
    "id": 25,
    "email": "bancha.zeyou@gmail.com",
    "address": {
        "district": "Watthana",
        "location": "6 Thong Lo Road",
        "province": "Bangkok",
        "postal_code": 10110
    },
    "full_name": "Bancha Kittisompong"
}
{
    "id": 19,
    "email": "chayapol.231@gmail.com",
    "address": {
        "district": "Hang Chat",
        "location": "160 Lampang Road",
        "province": "Lampang",
        "postal_code": 52190
    },
    "full_name": "Chayapol Komprach"
}
{
    "id": 4,
    "email": "dejwit.tt@gmail.com",
    "address": {
        "district": "Chiang Khan",
        "location": "150 Loei Road",
        "province": "Loei",
        "postal_code": 42110
    },
    "full_name": "Dejwit Tangjareonsakul"
}
{
    "id": 11,
    "email": "jack.maa@gmail.com",
    "address": {
        "district": "Bang Khen",
        "location": "88 Phahonyothin Road",
        "province": "Bangkok",
        "postal_code": 10220
    },
    "full_name": "Jack Maa"
}

```

In [35]:
c = Customer.objects.annotate(full_name=Concat('first_name', V(" "), 'last_name')).order_by('full_name').values('id', 'email', 'address', 'full_name')[:5]
for i in c:    
    print(json.dumps(i, indent=4, sort_keys=False))

{
    "id": 17,
    "email": "anantaya.deena@gmail.com",
    "address": {
        "district": "Yan Nawa",
        "location": "60 Thanon Chan Road",
        "province": "Bangkok",
        "postal_code": 10120
    },
    "full_name": "Anantaya Tontong"
}
{
    "id": 25,
    "email": "bancha.zeyou@gmail.com",
    "address": {
        "district": "Watthana",
        "location": "6 Thong Lo Road",
        "province": "Bangkok",
        "postal_code": 10110
    },
    "full_name": "Bancha Kittisompong"
}
{
    "id": 19,
    "email": "chayapol.231@gmail.com",
    "address": {
        "district": "Hang Chat",
        "location": "160 Lampang Road",
        "province": "Lampang",
        "postal_code": 52190
    },
    "full_name": "Chayapol Komprach"
}
{
    "id": 4,
    "email": "dejwit.tt@gmail.com",
    "address": {
        "district": "Chiang Khan",
        "location": "150 Loei Road",
        "province": "Loei",
        "postal_code": 42110
    },
    "full_name": "Dejwit Tangjareonsakul

### 3. aggregation - count(), sum(), avg()


3.1 ให้นักศึกษาหาค่าเฉลี่ยของราคาสินค้า (`Product.price`) ที่มีจำนวนคงเหลือ (`Product.remaining_amount`) มากกว่า 0 (0.2 คะแนน)

``` PYTHON
{'avg': Decimal('29308.000000000000')}
```


In [2]:
Product.objects.filter(remaining_amount__gt=0).aggregate(Avg("price"))

{'price__avg': Decimal('29308.000000000000')}

3.2 จงหาผลรวมราคา (`CartItem.product.price`) ที่อยู่ในตระกร้าสินค้าของเดือน `พฤษภาคม` (ดูจาก `Cart.create_date`) (0.2 คะแนน)

```PYTHON
{'sum': Decimal('9912555.00')}

```

In [7]:
CartItem.objects.filter(cart__create_date__month=5).aggregate(Sum('product__price'))

{'product__price__sum': Decimal('9912555.00')}

3.3 ให้นักศึกษานับจำนวนสินค้าที่อยู่ประเภท `Electronics`,  `Jewelry` และ ราคาของสินค้าอยู่ในช่วง 8,000.00 - 50,000.00 (0.2 คะแนน)

```
PRODUCT CATEGORY NAME: Electronics, PRODUCT COUNT: 6
PRODUCT CATEGORY NAME: Jewelry, PRODUCT COUNT: 1
```

In [23]:
p1 = Product.objects.filter(categories__name='Electronics').filter(price__gt=8000, price__lt=50000).count()
p2 = Product.objects.filter(categories__name='Jewelry').filter(price__gt=8000, price__lt=50000).count()

print(F"PRODUCT CATEGORY NAME: Electronics, PRODUCT COUNT: {p1}")
print(F"PRODUCT CATEGORY NAME: Jewelry, PRODUCT COUNT: {p2}")

PRODUCT CATEGORY NAME: Electronics, PRODUCT COUNT: 6
PRODUCT CATEGORY NAME: Jewelry, PRODUCT COUNT: 1


3.4 ให้นักศึกษามีการใช้ `Subquery` เพื่อหาคำสั่งซื้อ (`Order`) ล่าสุดของลูกค้า (`Customer`) แต่ละคน โดยมีการเรียงลำดับคำสั่งซื้อตามวันที่ที่สั่งซื้อ และแสดงผลข้อมูลตามตัวอย่างที่กำหนดไว้ (0.4 คะแนน)
```txt
# ตัวอย่างผลลัพธ์บางส่วน
CUSTOMER NAME: Sek Loso, ORDER DATE: 2024-05-13
CUSTOMER NAME: Warit Pititat, ORDER DATE: 2024-05-26
CUSTOMER NAME: Chayapol Komprach, ORDER DATE: 2024-05-26
CUSTOMER NAME: Kwantip Aekopas, ORDER DATE: 2024-05-28
CUSTOMER NAME: Kerikwan Laopong, ORDER DATE: 2024-05-30
CUSTOMER NAME: Pattanapoom Srisawangjan, ORDER DATE: 2024-05-30
CUSTOMER NAME: Manit Senapan, ORDER DATE: 2024-06-06
CUSTOMER NAME: Techathorn Sriwanwilai, ORDER DATE: 2024-06-07
CUSTOMER NAME: Anantaya Tontong, ORDER DATE: 2024-06-08
CUSTOMER NAME: Kittitorn Wilairat, ORDER DATE: 2024-06-11
...
```

In [20]:
order = Order.objects.filter(customer=OuterRef("pk")).order_by('-order_date')
c = Customer.objects.annotate(newest=Subquery(order.values('order_date')[:1]))
for i in c:
    print(F"CUSTOMER NAME: {i.first_name} {i.last_name}, ORDER DATE: {i.newest}")

CUSTOMER NAME: Panita Hongsakulpan, ORDER DATE: 2024-06-22
CUSTOMER NAME: Pakin Janpen, ORDER DATE: 2024-06-23
CUSTOMER NAME: Jenjira Sukanansarn, ORDER DATE: 2024-06-18
CUSTOMER NAME: Dejwit Tangjareonsakul, ORDER DATE: 2024-06-16
CUSTOMER NAME: Pong Sawadiwong, ORDER DATE: 2024-06-15
CUSTOMER NAME: Thitirat Sukkesorn, ORDER DATE: 2024-06-21
CUSTOMER NAME: Porntipa Pasakul, ORDER DATE: 2024-06-12
CUSTOMER NAME: Warit Pititat, ORDER DATE: 2024-05-26
CUSTOMER NAME: Sira Pititat, ORDER DATE: 2024-06-19
CUSTOMER NAME: Wanaporn Klabpetch, ORDER DATE: 2024-06-20
CUSTOMER NAME: Jack Maa, ORDER DATE: 2024-06-19
CUSTOMER NAME: Jakkapob Sopapak, ORDER DATE: 2024-06-18
CUSTOMER NAME: Kittitorn Wilairat, ORDER DATE: 2024-06-11
CUSTOMER NAME: Yanisa Nantapak, ORDER DATE: 2024-06-17
CUSTOMER NAME: stave klolas, ORDER DATE: 2024-06-22
CUSTOMER NAME: Peeraya Wichienchan, ORDER DATE: 2024-06-13
CUSTOMER NAME: Anantaya Tontong, ORDER DATE: 2024-06-08
CUSTOMER NAME: Sek Loso, ORDER DATE: 2024-05-13
CUST

### 4. one-to-one & one-to-many

4.1 จงสร้าง Order และ Payment ของลูกค้า Sek Loso จำนวน 2 รายการ โดยแต่ละ Order มีสินค้า 1 รายการ และชำระเงินด้วย Payment คนละรายการ จากนั้น เขียน query แสดงข้อมูล Order และ Payment ทั้งหมดของ Sek Loso (0.25 คะแนน)
```txt
# ตัวอย่างผลลัพธ์บางส่วน
CUSTOMER: Sek Loso, ORDER ID: 48, PRICE: 1699.00, DISCOUNT: 112.27, PRODUCT: Bird Cage
CUSTOMER: Sek Loso, ORDER ID: 59, PRICE: 1740.00, DISCOUNT: 140.81, PRODUCT: Microwave Oven
```

In [44]:
sek = Customer.objects.get(first_name='Sek', last_name="Loso")

product1 = Product.objects.get(name="Bird Cage")
product2 = Product.objects.get(name="Microwave Oven")

order1 = Order.objects.create(customer=sek, order_date=datetime.now().date())
order2 = Order.objects.create(customer=sek, order_date=datetime.now().date())

payment1 = Payment.objects.create(order=order1, payment_date=datetime.now().date(), price=1699, discount=112.27)
payment2 = Payment.objects.create(order=order2, payment_date=datetime.now().date(), price=1740, discount=140.81)

orderItem1 = OrderItem.objects.create(order=order1, product=product1, amount=1)
orderItem2 = OrderItem.objects.create(order=order2, product=product2, amount=1)

PaymentMethod.objects.create(payment=payment1, method='QR', price=1586.73)
PaymentMethod.objects.create(payment=payment2, method='CREDIT', price=1599.19)

orders = Order.objects.filter(customer=sek)

for order in orders:
    payment = Payment.objects.filter(order=order).first()
    order_item = OrderItem.objects.filter(order=order).first()

    print(F"CUSTOMER: {sek.first_name} {sek.last_name}, ORDER ID: {order.id}, PRICE: {payment.price}, DISCOUNT: {payment.discount}, PRODUCT: {order_item.product.name}")

CUSTOMER: Sek Loso, ORDER ID: 48, PRICE: 1699.00, DISCOUNT: 112.27, PRODUCT: Bird Cage
CUSTOMER: Sek Loso, ORDER ID: 59, PRICE: 1740.00, DISCOUNT: 140.81, PRODUCT: Microwave Oven
CUSTOMER: Sek Loso, ORDER ID: 185, PRICE: 1699.00, DISCOUNT: 112.27, PRODUCT: Bird Cage
CUSTOMER: Sek Loso, ORDER ID: 186, PRICE: 1740.00, DISCOUNT: 140.81, PRODUCT: Microwave Oven
CUSTOMER: Sek Loso, ORDER ID: 187, PRICE: 1699.00, DISCOUNT: 112.27, PRODUCT: Bird Cage
CUSTOMER: Sek Loso, ORDER ID: 188, PRICE: 1740.00, DISCOUNT: 140.81, PRODUCT: Microwave Oven
CUSTOMER: Sek Loso, ORDER ID: 189, PRICE: 1699.00, DISCOUNT: 112.27, PRODUCT: Bird Cage
CUSTOMER: Sek Loso, ORDER ID: 190, PRICE: 1740.00, DISCOUNT: 140.81, PRODUCT: Microwave Oven


4.2 ให้นักศึกษาดึงข้อมูล `Payment` ที่มี `PaymentMethod` ที่เป็น 'CREDIT' และมีราคาหลังหักส่วนลด (`Payment.price` - `Payment.discount`) ต่ำกว่า 500 และให้แสดงจำนวนของ `PaymentItem` ของแต่ละรายการ Payment (0.25 คะแนน)
```txt
# ตัวอย่างผลลัพธ์
PAYMENT ID: 40, AFTER_DISCOUNT: 493.06, ITEM COUNT: 2
PAYMENT ID: 13, AFTER_DISCOUNT: 448.54, ITEM COUNT: 2
```

In [4]:
p = Payment.objects.annotate(after_discount=F('price')-F('discount')).filter(after_discount__lt=500, paymentmethod__method='CREDIT').annotate(pitem=Count('paymentitem'))
for i in p:
    print(F'PAYMENT ID: {i.id}, AFTER_DISCOUNT: {i.after_discount}, ITEM COUNT: {i.pitem}')

PAYMENT ID: 40, AFTER_DISCOUNT: 493.06, ITEM COUNT: 2
PAYMENT ID: 13, AFTER_DISCOUNT: 448.54, ITEM COUNT: 2


### 5. many-to-many

5.1 ให้นักศึกษาค้นหาข้อมูลสินค้า (`Product`) ที่อยู่ในทั้งหมวดหมู่สินค้า "Information Technology" และ "Electronics" 10 รายการแรก (เรียงลำดับด้วย `Product.id`) และแสดงชื่อประเภทสินค้า (`ProductCategory`) (0.25 คะแนน)

ตัวอย่าง Output บางส่วน

```
PRODUCT ID: 1, PRODUCT NAME: Smartphone, PRODUCT CATEGORY: Information technology, Electronics
PRODUCT ID: 2, PRODUCT NAME: Laptop, PRODUCT CATEGORY: Information technology, Electronics
PRODUCT ID: 3, PRODUCT NAME: Smart TV, PRODUCT CATEGORY: Information technology, Electronics
```

In [3]:
p = Product.objects.filter(categories__name='Information Technology').filter(categories__name='Electronics').order_by('id')[:10]
for i in p:
    print(F"PRODUCT ID: {i.id}, PRODUCT NAME: {i.name}, PRODUCT CATEGORY: {', '.join(c.name for c in i.categories.all())}")

5.2 ให้นักศึกษาทำตามขั้นตอนดังนี้ (0.25 คะแนน)

1. เปลี่ยนชื่อหมวดหมู่สินค้า `Electronics` เป็น `Electronics and Toys` แทน
2. ย้ายสินค้าที่อยู่ใน `Toys and Games` ไปอยู่ในหมวดหมู่ `Electronics and Toys` และ ลบหมวดหมู่สินค้า `Toys and Games` 
3. ค้นหาว่าสินค้าที่มีประเภทสินค้าเป็น `Electronics and Toys` ทั้งหมดมีจำนวนเท่าไหร่

In [93]:
c = ProductCategory.objects.get(name="Electronics")
c.name = "Electronics and Toys"
c.save()

DoesNotExist: ProductCategory matching query does not exist.

In [39]:
c1 = ProductCategory.objects.get(name="Toys and Games")
c2 = ProductCategory.objects.get(name="Electronics and Toys")

p1 = Product.objects.filter(categories=c1)

for i in p1:
    i.categories.add(c2)
    i.categories.remove(c1)

c1.delete()

(1, {'shop.ProductCategory': 1})

In [37]:
p = Product.objects.filter(categories__name='Electronics and Toys').count()
p

30